# Amazon Review Clustering

Miloy Ajmera (majmera@andrew.cmu.edu) / 17-634: Applied Machine Learning, Spring 2023

In [ ]:
import os
import sklearn
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import random

import seaborn as sns  # We'll mostly use seaborn, a stats visualization library
import matplotlib.pyplot as plt  # Seaborn wrap Matplotlib for visualization. We'll occasionaly want to make custom changes to the plots.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/Othercomputers/My PC/Carnegie Mellon University Notes/Semester 3 - Spring 2023/Applied Machine Learning/Homework Assignments/Amazon/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 amazon						       list.xlsx
 amazon.model					       questions3.txt
'Amazon Review Clustering - MetaData - Backup.ipynb'   topic_model1.model
'Amazon Review Clustering - MetaData.ipynb'	       topic_model2.model
'Amazon Review Clustering - Small Sample.ipynb'        topic_model3.model
'Copy of Amazon Review Clustering - MetaData.ipynb'    topic_model6.model
 counts.csv


In [ ]:
filepath = "/content/drive/Othercomputers/My PC/Carnegie Mellon University Notes/Semester 3 - Spring 2023/Applied Machine Learning/Homework Assignments/Amazon/"

## Load the Amazon Review dataset.

In [ ]:
# get the contents and size of the json file - 2 mins 45 secs
filename = 'amazon/meta_Home_and_Kitchen.json'
# Load the dataframes
num_lines = sum(1 for line in open(filepath + filename))
print(num_lines)

1300540


In [ ]:
# get ranges - 0 secs
df_range = range(0, num_lines)
ranges = []
for i in range(0, len(df_range), int(len(df_range)/10)) :
  ranges.append(df_range[i:i+int(len(df_range)/10)])

ranges = ranges[:-1]
print(ranges)

[range(0, 130054), range(130054, 260108), range(260108, 390162), range(390162, 520216), range(520216, 650270), range(650270, 780324), range(780324, 910378), range(910378, 1040432), range(1040432, 1170486)]


In [ ]:
# understand data - 2 mins  54 secs
import csv
import json
json_file = open(filepath + filename, "r")
# for ri, single_range in enumerate(ranges):
data = []
for i, line in enumerate(json_file):
    if i in ranges[0]:
      data.append(json.loads(line))
df = pd.DataFrame.from_records(data)
df.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Home & Kitchen, Kitchen & Dining, Dining & En...",,[It was a time honored tradition among the ear...,,You Are Special Today Red Plate [With Red Pen],"[B0001XR2F2, B01LY51HUN, B07CXZ9C5B, 0310258952]",[],,Waechtersbach USA,[],"[>#39,665 in Kitchen & Dining (See Top 100 in ...","[B0001XR2F2, B00MOFKX1A, B07G3LN13B, B07CYXMFF...",Amazon Home,"class=""a-bordered a-horizontal-stripes a-spa...","October 8, 2006",$37.00,0001487795,NaN
1,"[Home & Kitchen, Home Dcor, Candles & Holders,...",,[VICKS INHALER relieves stuffy noses helps sin...,,Vicks Inhaler Relief for Cold Sinus Nasal Cong...,[],[],,Vicks,[],"[>#1,763,185 in Home & Kitchen (See Top 100 in...",[B00UPCRZEC],Amazon Home,,,$4.05,0002020300,NaN
2,"[Home & Kitchen, Kitchen & Dining, Dining & En...",,"[16 oz squeeze bottle, 1 lb.]",,Artistic Churchware Communion Cup Filler: RW525,[],[],,Artistic Churchware,"[Religious Supply Center, RW-525, Communion Cu...","[>#2,127,003 in Home & Kitchen (See Top 100 in...","[B00C9J79TA, B00XB3ZG5M, B0714K8MSR, 078472530...",Amazon Home,,,$12.48,0006564224,NaN
3,"[Home & Kitchen, Bath, Bathroom Accessories]",,[The only soap in the world with a unique comb...,,4 BARS! Mysore Sandal Soap 70grams FAST SHIPPING,[],[https://images-na.ssl-images-amazon.com/image...,,Mysore,[],"[>#6,942,841 in Home & Kitchen (See Top 100 in...",[B001G7PZB0],Amazon Home,,,$22.00,0009046461,NaN
4,"[Home & Kitchen, Home Dcor, Home Fragrance, In...",,"[Divya Arogya vati improves health, It has nat...",,AROGYA VATI (40gm) by popeye seller,[],[],,Patanjali,[],"[>#3,103,399 in Home & Kitchen (See Top 100 in...","[B075M91MX3, 0258798785, 6565652554]",Amazon Home,,,$5.10,0234937912,NaN


### Create Filter based on the categories

In [ ]:
# take list of all categories from meta data - 3 mins 52 secs
import csv
import json
from collections import Counter
json_file = open(filepath + filename, "r")
data = []
for i, line in enumerate(json_file):
      json_object = json.loads(line)
      data.extend(json_object['category'])
df = pd.DataFrame(data, columns=['category'])
df.head()

,category
0,Home & Kitchen
1,Kitchen & Dining
2,Dining & Entertaining
3,Dinnerware
4,Plates


In [ ]:
from pandas.core.indexes import category
# Take the specific categories whose counts are more than 50,000 - 1 sec
category_list = df['category'].value_counts()
category_list = category_list.iloc[9:12]
category_list


Storage & Organization    78111
Furniture                 75419
Bath                      53144
Name: category, dtype: int64

In [ ]:
# get the total of values
total = category_list.sum()
print("Percentage of " + str(total) + " categories picked out of " + str(num_lines) + " = " + str(total * 100 / num_lines))
list_of_categories = category_list.index.tolist()
list_of_categories

Percentage of 206674 categories picked out of 1300540 = 15.891398957356175


['Storage & Organization', 'Furniture', 'Bath']

In [ ]:
# Take the asin's for the category selected - 4 mins 0 secs
json_file = open(filepath + filename, "r")
data_list = []
for i, line in enumerate(json_file):
        json_object = json.loads(line)
        list_of_categories = json_object['category']
        data = {}
        for each_category in list_of_categories:
            if each_category in category_list:
              data['asin'] = json_object['asin']
              data['category'] = json_object['category']
              data_list.append(data)
              break
data_list[:10]

[{'asin': '0009046461',
  'category': ['Home & Kitchen', 'Bath', 'Bathroom Accessories']},
 {'asin': '0555667502',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Travel & To-Go Food Containers',
   'Lunch Bags']},
 {'asin': '0938985329',
  'category': ['Home & Kitchen',
   'Storage & Organization',
   'Home Storage Hooks']},
 {'asin': '1581174292',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Food Storage',
   'Cookie Jars']},
 {'asin': '1605534137',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Food Storage',
   'Food Storage & Organization Sets']},
 {'asin': '1616619813',
  'category': ['Home & Kitchen', 'Furniture', "Kids' Furniture"]},
 {'asin': '163375233X',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Travel & To-Go Food Containers',
   'Reusable Grocery Bags']},
 {'asin': '1909558672',
  'category': ['Home & 

In [ ]:
#Get unique list of asins and put them in a list
asins = []
for item in data_list:
    asins.append(item['asin'])
unique_asins = set(asins)
print("Percentage of " + str(len(unique_asins)) + " asins picked out of " + str(num_lines) + " = " + str(len(unique_asins) * 100 / num_lines))


Percentage of 205126 asins picked out of 1300540 = 15.772371476463622


### Use Home and Kitchen JSON file and map the corresponding asins

In [ ]:
# get the contents and size of the json file - 1 min 10 secs
filename = 'amazon/Home_and_Kitchen.json'
# Load the dataframes
num_lines = sum(1 for line in open(filepath + filename))
print(num_lines)

21928568


In [ ]:
# understand data - 11 mins
import csv
import json
with open(filepath + 'amazon/asin-data.json', 'w') as writer_file:
  json_file = open(filepath + filename, "r")
  data = []
  for i, line in enumerate(json_file):
    data = json.loads(line)
    if data['asin'] in unique_asins:
      json.dump(data, writer_file)
      writer_file.write('\n')
  
# df = pd.DataFrame.from_records(data)
# df.head()


### Read the saved asins json file

In [ ]:
# get the contents and size of the json file
filename = 'amazon/asin-data.json'
# Load the dataframes
num_asin_lines = sum(1 for line in open(filepath + filename))
print(num_asin_lines)

4681185


In [ ]:
print("Percentage of " + str(num_asin_lines) + " asins picked out of " + str(num_lines) + " = " + str(num_asin_lines * 100 / num_lines))

Percentage of 4681185 asins picked out of 21928568 = 21.347426790477154


In [ ]:
# import random
# random_list = random.sample(range(num_asin_lines), int(num_asin_lines/10))
# random_set = set(random_list)
# print(len(random_set))
# print("Percentage of " + str(len(random_set)) + " asins picked out of " + str(num_lines) + " = " + str(len(random_set) * 100 / num_lines))

In [ ]:
import json
import pandas as pd
# extract data into a dataframe - 2 mins
json_file = open(filepath + 'amazon/asin-data.json', "r")
data = []
for i, line in enumerate(json_file):
      # if i in random_set:
    data.append(json.loads(line))
df_filtered = pd.DataFrame.from_records(data)
df_filtered.head()


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"08 9, 2013",ACQGF9PXBB35M,1581174292,{'Format:': ' Hardcover'},Ed Mirabal,Its a nice variation on the kids book and the ...,Kids book,1376006400,NaN,NaN
1,5.0,True,"02 2, 2013",A3P565UNR58SPG,1581174292,{'Format:': ' Hardcover'},Monnie D. Brewster,"The book teaches many things, how to count, to...",Very good for kid and we love it too.,1359763200,NaN,NaN
2,5.0,True,"01 12, 2013",A3LT7ZV6LZZF7N,1581174292,{'Format:': ' Hardcover'},tetani,Purchased this book as a gift for the grandchi...,Fun book,1357948800,NaN,NaN
3,5.0,True,"11 29, 2012",A13S4950ZDHDQG,1581174292,{'Format:': ' Hardcover'},Kindle Customer,We read this EVERY day at daycare. Santa has ...,My children's favorite book!,1354147200,NaN,NaN
4,3.0,True,"07 22, 2012",A13QQN9O57I2HO,1581174292,{'Format:': ' Hardcover'},L,This book is cute my 18 month old likes it. I...,Cute.,1342915200,NaN,NaN


In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4681185 entries, 0 to 4681184
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   verified        bool   
 2   reviewTime      object 
 3   reviewerID      object 
 4   asin            object 
 5   style           object 
 6   reviewerName    object 
 7   reviewText      object 
 8   summary         object 
 9   unixReviewTime  int64  
 10  vote            object 
 11  image           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 397.3+ MB


In [ ]:
print("Percentage of " + str(len(df_filtered)) + " asins picked out of " + str(num_lines) + " = " + str(len(df_filtered) * 100 / num_lines))

Percentage of 4681185 asins picked out of 21928568 = 21.347426790477154


### Lets filter the dataset to provide reviews which should have meaningful texts

In [ ]:
df_filtered = df_filtered[(df_filtered['verified'] == True) & (df_filtered['vote'].notna()) & (df_filtered['overall'] > 4.0)]
df_filtered.to_json(filepath + 'amazon/asin-data-filtered.json', orient='records')
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274678 entries, 16 to 4681169
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         274678 non-null  float64
 1   verified        274678 non-null  bool   
 2   reviewTime      274678 non-null  object 
 3   reviewerID      274678 non-null  object 
 4   asin            274678 non-null  object 
 5   style           169770 non-null  object 
 6   reviewerName    274665 non-null  object 
 7   reviewText      274234 non-null  object 
 8   summary         274596 non-null  object 
 9   unixReviewTime  274678 non-null  int64  
 10  vote            274678 non-null  object 
 11  image           44230 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 33.5+ MB


In [ ]:
print("Percentage of " + str(len(df_filtered)) + " asins picked out of " + str(num_lines) + " = " + str(len(df_filtered) * 100 / num_lines))

Percentage of 274678 asins picked out of 21928568 = 1.2526034531757841


### Take the filtered JSON which is around 1% of the data and then work on it

In [ ]:
# extract the dataframe into a new dataframe
df_filtered = pd.read_json(filepath + 'amazon/asin-data-filtered.json', orient='records')
df_filtered = df_filtered[['reviewText']]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274678 entries, 0 to 274677
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   reviewText  274234 non-null  object
dtypes: object(1)
memory usage: 2.1+ MB


### Now perform LDA on the extracted dataset

In [ ]:
from gensim import models, similarities
from gensim.corpora.dictionary import Dictionary
# Plus a few other assorted inputs.
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Clean the data
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: np.nan if len(str(x)) < 2 else x)
df_filtered = df_filtered.dropna()
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: x.lower())
df_filtered.head()

,reviewText
0,great mat for theme or general play. it doesn'...
1,these are just what i was needing to keep my p...
2,i absolutely love these!! they are the perfect...
3,"<div id=""video-block-r10wxkxpgc8kma"" class=""a-..."
4,"for what i needed, these are great little cont..."


In [ ]:
# Removing stop words
import pandas as pd
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: ' '.join([word for word in str(x).split() if str(word) not in (stop_words)]))
df_filtered.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,reviewText
0,great mat theme general play. really keep ship...
1,needing keep pills supplements ready taking mo...
2,absolutely love these!! perfect size many thin...
3,"<div id=""video-block-r10wxkxpgc8kma"" class=""a-..."
4,"needed, great little containers. reviewers say..."


In [ ]:
# Here we start from full sentences again to help the NLP library understand the types of words. 
# Note: this can take quite a while! - 5 mins
from gensim.utils import tokenize
def tokenize_text(text):
    return list(tokenize(text))

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(word) for word in tokenize_text(text)]
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lemmatize_text)
df_filtered.head()

,reviewText
0,"[great, mat, theme, general, play, really, kee..."
1,"[needing, keep, pill, supplement, ready, takin..."
2,"[absolutely, love, these, perfect, size, many,..."
3,"[div, id, video, block, r, wxkxpgc, kma, class..."
4,"[needed, great, little, container, reviewer, s..."


In [ ]:
# # We'd typically start by tokenizing the data 
# from gensim.utils import tokenize
# def tokenize_text(text):
#     return list(tokenize(text))
# df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: tokenize_text(x))
# df_filtered.head()


In [ ]:
# # And then stem all words.
# from gensim.parsing.porter import PorterStemmer

# stemmer = PorterStemmer()

# def stem_text(word):
#     return stemmer.stem(word)

# df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: [stemmer.stem(word) for word in x])
# df_filtered.head()

In [ ]:
# Create a corpus from a list of texts.
# The dictionary just extracts and numbers each distinct word.
dictionary = Dictionary(df_filtered['reviewText'])
# A corpus is a sparse datastore containing the number of times each word appears in each document.
corpus = [dictionary.doc2bow(text) for text in df_filtered['reviewText']]


In [ ]:
# Print a sample of dictionary items.
top_words_in_doc_0 = sorted(corpus[0], key=lambda e: e[1], reverse=True)[:10]
for word_index, count in top_words_in_doc_0:
  print(f'{dictionary[word_index]}   \t\t index: {word_index}\tcount: {count:,}')

ship   		 index: 17	count: 2
amazing   		 index: 0	count: 1
anything   		 index: 1	count: 1
backdrop   		 index: 2	count: 1
easier   		 index: 3	count: 1
everyday   		 index: 4	count: 1
game   		 index: 5	count: 1
general   		 index: 6	count: 1
great   		 index: 7	count: 1
identical   		 index: 8	count: 1


In [ ]:
# Let's start with a simple TF-IDF model. Worth noting: SKLearn also has a Tfidf vectorizer.
model = models.TfidfModel(corpus)

In [ ]:
# What are the most _salient_ words in document 0?
top_tfidf_words = sorted(model[corpus[0]], key=lambda e: e[1], reverse=True)[:10]
for word_index, score in top_tfidf_words:
  print(f'{dictionary[word_index]}   \t\t index: {word_index} \t\t tf-idf: {score:.2f}')

# What's different from the previous ranking? What do you think the topic label might be?

ship   		 index: 17 		 tf-idf: 0.42
backdrop   		 index: 2 		 tf-idf: 0.32
wing   		 index: 23 		 tf-idf: 0.27
mouse   		 index: 13 		 tf-idf: 0.24
general   		 index: 6 		 tf-idf: 0.24
identical   		 index: 8 		 tf-idf: 0.24
theme   		 index: 22 		 tf-idf: 0.23
everyday   		 index: 4 		 tf-idf: 0.21
game   		 index: 5 		 tf-idf: 0.20
sliding   		 index: 20 		 tf-idf: 0.20


## Create LDA Models #1

In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 9 mins

# Let's start with 20 topics.
num_topics = 20
model = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=2, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  one well love would good product price great towel quality
1:  holder clean get like mat sink work water one use
2:  box one like comfortable i it foam sleep bed mattress
3:  like hang hold closet great use one clothes hanger bag
4:  inside fit day carry keep cold pack box bag lunch
5:  utf ref ie hook class product normal b link a
6:  put use basket easy fit sturdy hold space rack shelf
7:  sofa seat tv top chair wood stand couch leg table
8:  well like hook liner look color soap bathroom curtain shower
9:  block makeup fit pair drawer video image organizer i shoe
10:  exactly looking fit nice color room love perfect great look
11:  one work well great fit lid size use food container
12:  seat it ve like one back comfortable m i chair
13:  beautiful look assemble sturdy put together great well table easy
14:  plug work moth oil ball product hole cord power screw
15:  ve time get open trash lid can one i bag
16:  installed two easy screw product install wall one door cabinet
17:  s

In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.47044475978617417

In [ ]:
with open(filepath + 'topic_model1.model', 'wb') as f:
    pickle.dump(model,f)

## Create LDA Models #2

In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 15 mins

# Let's start with 20 topics.
num_topics = 20
model2 = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=4, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model2.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  tub clean one dish work dispenser water stool mat soap
1:  perfect assemble love look sturdy room great together put easy
2:  amazon x normal product video b link class image a
3:  bath great well i use paper dry wash soft towel
4:  hold hanging i use one hook hang clothes closet hanger
5:  great wall product i cup brush work rod one shower
6:  quality amazon price delivery shipping arrived frame great product bed
7:  cushion sofa i great leg sit back seat comfortable chair
8:  top one part assembly put instruction wood together screw piece
9:  fit top one inch wall unit desk tv stand shelf
10:  like s thing get m box it i review one
11:  carry great hold size laundry fit trash use one bag
12:  drawer great item storage perfect small cabinet fit kitchen space
13:  heavy use glass one side bottle fit top hold rack
14:  jar open container seal keep use one plastic i lid
15:  great gift one old absolutely year daughter bought it love
16:  one fit work use great size box food lunch con

In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model2, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.4937512092375761

In [ ]:
with open(filepath + 'topic_model2.model', 'wb') as f:
    pickle.dump(model2,f)

## Create LDA Models #3

In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 14 mins

# Let's start with 10 topics.
num_topics = 10
model3 = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=4, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model3.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  company shipping day delivery box arrived review amazon product one
1:  perfect well beautiful nice table quality color love great look
2:  would bathroom plastic hook i wall like one curtain shower
3:  night comfortable frame like foam sleep it i mattress bed
4:  hold well size basket fit great lid use one bag
5:  storage use hanger closet one hold rack shelf fit space
6:  stool couch leg sit like i seat back comfortable chair
7:  daughter great s old i year bought one it love
8:  assemble wood took assembly sturdy look piece easy put together
9:  like work great one food i use lunch container towel


In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model3, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.4720916348534958

In [ ]:
with open(filepath + 'topic_model3.model', 'wb') as f:
    pickle.dump(model3,f)

## Create LDA Models #4

In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 29 mins

# Let's start with 10 topics.
num_topics = 10
model4 = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=8, workers=2, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model4.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  part two hole bottom piece side one shelf screw top
1:  one love look great like color bathroom curtain shower towel
2:  back frame comfortable foam like sleep i it mattress bed
3:  sturdy it well quality love put together look great easy
4:  it seat look room great color love comfortable table chair
5:  sturdy closet i hanger great space use hold rack one
6:  well look small it kitchen great size fit perfect love
7:  like work food i lid one use lunch bag container
8:  normal i product b link video class image a bag
9:  day time quality would price i amazon product review one


In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model4, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.4850476512741956

In [ ]:
with open(filepath + 'topic_model4.model', 'wb') as f:
    pickle.dump(model4,f)

## Create LDA Models #5

In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 27 mins

# Let's start with 10 topics.
num_topics = 20
model5 = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, alpha = 0.5, passes=4, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model5.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  it wife home love product great work keep well made
1:  look instruction furniture wood assembly took easy piece together put
2:  room size inch guest spring bedroom headboard box frame bed
3:  t thing like love get s ve it m i
4:  day inside work size fit use pack box lunch bag
5:  easily worth hold fold basket laundry handle clothes use heavy
6:  arm work expectation liner office back chair shower desk curtain
7:  size storage hold love kitchen perfectly perfect small space fit
8:  cup make seal set plastic size use food lid container
9:  pair b normal class link x product hook shoe a
10:  perfect room sit seat old great it comfortable love chair
11:  thick like sink wash color bath soft soap bathroom towel
12:  day customer would order service received company one review product
13:  smell like comfortable back night firm foam sleep bed mattress
14:  side rack closet hang top wall shelf hole hanger screw
15:  worked used tub use great water get work clean shower
16:  color recom

In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model5, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.4924300384342226

In [ ]:
with open(filepath + 'topic_model5.model', 'wb') as f:
    pickle.dump(model5,f)

## Create LDA Models #6

In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 27 mins

# Let's start with 10 topics.
num_topics = 20
model6 = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, alpha = 1, passes=4, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model6.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  product price bought it work well buy good really great
1:  think well one i great happy purchase love it review
2:  use them it bought great hold like perfect one love
3:  book unit well nice easy together sturdy set shelf put
4:  tub liner mat product hook a soap bathroom curtain shower
5:  x great top drawer value type amazon small i size
6:  use space hold one hanging clothes hang closet hanger rack
7:  paper hand like thick use wash color dry soft towel
8:  small look perfect bedroom it install product wall love easy
9:  bottom like use clean dish holder counter sink cabinet kitchen
10:  look highly easy love put exactly would recommend great quality
11:  price product amazon company ordered received delivery shipping color arrived
12:  office sitting height foot stool sit seat back comfortable chair
13:  comfortable like i night firm frame foam sleep mattress bed
14:  hold item perfectly organizer space shoe one door well fit
15:  solid piece assemble furniture easy wood room

In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model6, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.48016399676398674

In [ ]:
with open(filepath + 'topic_model6.model', 'wb') as f:
    pickle.dump(model6,f)

## Create LDA Models #7

In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 27 mins

# Let's start with 10 topics.
num_topics = 20
model7 = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=8, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model7.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  price buy happy bought would one m i it love
1:  excellent thank amazon image great seller company customer service product
2:  great sturdy mattress headboard box together easy put frame bed
3:  memory like back firm night comfortable sleep foam bed mattress
4:  clean easy use one paper lid bag open can trash
5:  part put top hole assembly wood instruction together piece screw
6:  great stool love couch sit back desk seat comfortable chair
7:  ie nbsp hook product normal x b link class a
8:  piece came time delivery shipping day arrived one amazon box
9:  great nice soft like love bathroom color curtain shower towel
10:  would that m it s review get thing like i
11:  bought last they re time month year ve one i
12:  use one holder dispenser work water cup sink dish soap
13:  bottle fit shelf wall hold wine rod glass tv stand
14:  sturdy nice love together well quality easy table great look
15:  size makeup hold box love one organizer fit drawer shoe
16:  plastic keep jar great sea

In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model7, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.512073200304486

In [ ]:
with open(filepath + 'topic_model7.model', 'wb') as f:
    pickle.dump(model7,f)